In [1]:
# Random Imports
import os
import sys
import yaml
from pathlib import Path
sys.path.append('/storage/vbutoi/projects')
sys.path.append('/storage/vbutoi/libraries')
sys.path.append('/storage/vbutoi/projects/ESE')
sys.path.append('/storage/vbutoi/projects/UniverSegDev')

# Regular schema dictates that we put DATAPATH
os.environ['DATAPATH'] = ':'.join((
       '/storage/vbutoi/datasets',
))
os.environ['WANDB_NOTEBOOK_NAME'] = 'calibrate.ipynb'

# IonPy imports
from ionpy.util import Config

%load_ext yamlmagic
%load_ext autoreload
%autoreload 2

In [ ]:
%%yaml default_cfg

experiment:
  restart: True # Important, otherwise we don't load the model.

train:
  epochs: '?' # How much longer to train for.
  load_chkpt: '?' # Which model do we load
  pretrained_dir: '?' # Which runs are we restarting

In [ ]:
%%yaml callbacks_cfg

callbacks:
  step:
    - ese.callbacks.ShowPredictions
  epoch:
    - ese.callbacks.WandbLogger
    - ionpy.callbacks.ETA
    - ionpy.callbacks.JobProgress
    - ionpy.callbacks.TerminateOnNaN
    - ionpy.callbacks.PrintLogged
    - ionpy.callbacks.ModelCheckpoint:
        monitor: dice_score
        phase: val

In [ ]:
%%yaml experiment_cfg 

name: "ISLES_3D_LTS_Bs8_lr1e-4_RESTARTED" # We will treat this as a NEW EXPERIMENT.

train:
    epochs: 1000
    load_chkpt: 'last'
    pretrained_dir: "/storage/vbutoi/scratch/ESE/calibration/09_27_24_ISLES_3D_LTS_Bs8_lr1e-4"

In [5]:
from ese.analysis.analysis_utils.submit_utils import get_ese_restart_configs

# Get the configs for the different runs.
base_cfg = Config(default_cfg).update([callbacks_cfg])

updated_base_cfg, restart_cfgs = get_ese_restart_configs(
    exp_cfg=experiment_cfg,
    base_cfg=base_cfg
)

In [ ]:
len(restart_cfgs)

## Running Jobs

In [ ]:
from ese.experiment import run_ese_exp, submit_ese_exps, CalibrationExperiment, PostHocExperiment

In [8]:
# ####### Run individual jobs
# run_ese_exp(
#     config=restart_cfgs[0], 
#     experiment_class=CalibrationExperiment,
#     run_name='debug',
#     show_examples=True,
#     gpu='0'
# )

In [ ]:
#### Run Batch Jobs
submit_ese_exps(
    group="restarted",
    base_cfg=updated_base_cfg,
    exp_cfg=experiment_cfg,
    config_list=restart_cfgs,
    experiment_class=PostHocExperiment,
    track_wandb=True,
    available_gpus=['5', '6', '7']
)